In [1]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


Сейчас мы поработаем со встроенным датасетом attitude. Рассмотрим две модели

```
model_full <- lm(rating ~ ., data = attitude) 

model_null <- lm(rating ~ 1, data = attitude)
```

model_full - модель, которая предсказывает значение переменной рейтинг (rating) в зависимости от всех остальных переменных в данном датасете.

model_null - модель, в которой нет ни одного предиктора, а есть только intercept. Значение intercept - это просто среднее значение зависимой переменной. Соответственно, модель предоставляет нам информацию только о том, отличается ли это среднее от нуля.

Как говорилось в лекции, функция step позволяет нам подобрать модель с оптимальным количеством предикторов. С помощью аргумента scope мы можем задать пространство моделей с разным числом предикторов, в котором будет происходить поиск оптимального набора предикторов. Самый простой путь - задать границы возможных моделей с помощью нулевой и полной моделей.
```
scope = list(lower = model_null, upper = model_full)
```
Аргумент direction позволяет задать направление поиска.

Первый аргумент (object) задаёт начальную модель, с которой начинается поиск. Обратите внимание на то, что при разных значениях аргумента direction нужно использовать разные начальные модели.

Функция step возвращает оптимальную модель.

Итак, задача! C помощью функции step найдите оптимальную модель для предсказания rating в датасете attitude. Model_full и model_null уже созданы. Сохраните команду с функцией step в переменную ideal_model.

In [5]:
%%R 
str(attitude)

'data.frame':	30 obs. of  7 variables:
 $ rating    : num  43 63 71 61 81 43 58 71 72 67 ...
 $ complaints: num  51 64 70 63 78 55 67 75 82 61 ...
 $ privileges: num  30 51 68 45 56 49 42 50 72 45 ...
 $ learning  : num  39 54 69 47 66 44 56 55 67 47 ...
 $ raises    : num  61 63 76 54 71 54 66 70 71 62 ...
 $ critical  : num  92 73 86 84 83 49 68 66 83 80 ...
 $ advance   : num  45 47 48 35 47 34 35 41 31 41 ...


In [0]:
%%R 
model_full <- lm(rating ~ ., data = attitude) 

model_null <- lm(rating ~ 1, data = attitude)

In [3]:
%%R 
model_full


Call:
lm(formula = rating ~ ., data = attitude)

Coefficients:
(Intercept)   complaints   privileges     learning       raises     critical  
   10.78708      0.61319     -0.07305      0.32033      0.08173      0.03838  
    advance  
   -0.21706  



In [4]:
%%R 
model_null


Call:
lm(formula = rating ~ 1, data = attitude)

Coefficients:
(Intercept)  
      64.63  



In [9]:
%%R 
new_model <- step(model_full, scope = list(lower = model_null, upper = model_full), 
                  direction = c("both", "backward", "forward"))
new_model

Start:  AIC=123.36
rating ~ complaints + privileges + learning + raises + critical + 
    advance

             Df Sum of Sq    RSS    AIC
- critical    1      3.41 1152.4 121.45
- raises      1      6.80 1155.8 121.54
- privileges  1     14.47 1163.5 121.74
- advance     1     74.11 1223.1 123.24
<none>                    1149.0 123.36
- learning    1    180.50 1329.5 125.74
- complaints  1    724.80 1873.8 136.04

Step:  AIC=121.45
rating ~ complaints + privileges + learning + raises + advance

             Df Sum of Sq    RSS    AIC
- raises      1     10.61 1163.0 119.73
- privileges  1     14.16 1166.6 119.82
- advance     1     71.27 1223.7 121.25
<none>                    1152.4 121.45
+ critical    1      3.41 1149.0 123.36
- learning    1    177.74 1330.1 123.75
- complaints  1    724.70 1877.1 134.09

Step:  AIC=119.73
rating ~ complaints + privileges + learning + advance

             Df Sum of Sq    RSS    AIC
- privileges  1     16.10 1179.1 118.14
- advance     1     61.6

Сравните полную модель из предыдущего степа и оптимальную модель с помощью функции anova. Введите получившееся F-значение.

Разделителем дробной и целой части в ответе должна быть запятая.

In [12]:
%%R 
a_f <- anova(model_full, new_model)
a_f

Analysis of Variance Table

Model 1: rating ~ complaints + privileges + learning + raises + critical + 
    advance
Model 2: rating ~ complaints + learning
  Res.Df    RSS Df Sum of Sq      F Pr(>F)
1     23 1149.0                           
2     27 1254.7 -4   -105.65 0.5287 0.7158


In [16]:
%%R 
a_f$F

[1]        NA 0.5287028


Напоследок потренируемся в эффективном написании формул. В этой задаче будем работать со встроенным датасетом LifeCycleSavings. Попытаемся предсказать значение sr на основе всех остальных переменных в этом датасете. Вспомните способы сокращения формул и напишите команду, которая создаёт линейную регрессию с главными эффектами и всеми возможными взаимодействиями второго уровня. Сохраните модель в переменную model.

In [17]:
%%R 
lm(sr ~ (.)^2, data = LifeCycleSavings)


Call:
lm(formula = sr ~ (.)^2, data = LifeCycleSavings)

Coefficients:
(Intercept)        pop15        pop75          dpi         ddpi  pop15:pop75  
  9.204e+00   -6.423e-02    3.886e+00    3.200e-04    1.126e+00   -9.288e-02  
  pop15:dpi   pop15:ddpi    pop75:dpi   pop75:ddpi     dpi:ddpi  
 -9.644e-07   -1.151e-02   -8.923e-04   -2.863e-01    7.613e-04  

